<a href="https://colab.research.google.com/github/MatoKamenicky/PySpark-basics/blob/main/PySpark_basics_Part5_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler

In [3]:
spark = SparkSession.builder.appName('ml').getOrCreate()

In [7]:
training = spark.read.csv('/content/drive/MyDrive/code/pyspark/test1.csv', header=True, inferSchema=True)

In [8]:
training.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [10]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [9]:
training.columns

['Name', 'age', 'Experience', 'Salary']

In [13]:
featureassembler = VectorAssembler(inputCols=["age", "Experience"], outputCol="Independent Features")

In [14]:
output = featureassembler.transform(training)

In [15]:
output.show()

+---------+---+----------+------+--------------------+
|     Name|age|Experience|Salary|Independent Features|
+---------+---+----------+------+--------------------+
|    Krish| 31|        10| 30000|         [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|          [30.0,8.0]|
|    Sunny| 29|         4| 20000|          [29.0,4.0]|
|     Paul| 24|         3| 20000|          [24.0,3.0]|
|   Harsha| 21|         1| 15000|          [21.0,1.0]|
|  Shubham| 23|         2| 18000|          [23.0,2.0]|
+---------+---+----------+------+--------------------+



In [16]:
finalized_data = output.select(["Independent Features", "Salary"])

In [17]:
finalized_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [18]:
from pyspark.ml.regression import LinearRegression

In [27]:
train, test = finalized_data.randomSplit([0.75, 0.25])

In [28]:
regressor = LinearRegression(featuresCol='Independent Features', labelCol='Salary')
regressor = regressor.fit(train)

In [29]:
regressor.coefficients

DenseVector([-1363.6364, 3181.8182])

In [30]:
regressor.intercept

40454.54545453764

In [31]:
pred = regressor.evaluate(test)

In [32]:
pred.predictions.show()

+--------------------+------+------------------+
|Independent Features|Salary|        prediction|
+--------------------+------+------------------+
|          [23.0,2.0]| 18000|15454.545454545725|
|          [24.0,3.0]| 20000| 17272.72727272748|
|          [29.0,4.0]| 20000|13636.363636365342|
+--------------------+------+------------------+



In [33]:
pred.meanAbsoluteError, pred.meanSquaredError

(3878.787878787151, 18137741.04682388)